# क्लाउड में डेटा साइंस: "Azure ML SDK" का तरीका

## परिचय

इस नोटबुक में, हम सीखेंगे कि Azure ML SDK का उपयोग करके मॉडल को ट्रेन, डिप्लॉय और कंज्यूम कैसे करें।

पूर्व-आवश्यकताएँ:
1. आपने एक Azure ML वर्कस्पेस बनाया है।
2. आपने [हार्ट फेलियर डेटासेट](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) को Azure ML में लोड किया है।
3. आपने इस नोटबुक को Azure ML स्टूडियो में अपलोड किया है।

अगले चरण हैं:

1. एक मौजूदा वर्कस्पेस में एक एक्सपेरिमेंट बनाएं।
2. एक कंप्यूट क्लस्टर बनाएं।
3. डेटासेट लोड करें।
4. AutoMLConfig का उपयोग करके AutoML को कॉन्फ़िगर करें।
5. AutoML एक्सपेरिमेंट चलाएं।
6. परिणामों का विश्लेषण करें और सबसे अच्छा मॉडल प्राप्त करें।
7. सबसे अच्छे मॉडल को रजिस्टर करें।
8. सबसे अच्छे मॉडल को डिप्लॉय करें।
9. एंडपॉइंट का उपयोग करें।

## Azure Machine Learning SDK-विशिष्ट इंपोर्ट्स


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## कार्यक्षेत्र प्रारंभ करें
सहेजे गए कॉन्फ़िगरेशन से एक कार्यक्षेत्र ऑब्जेक्ट प्रारंभ करें। सुनिश्चित करें कि कॉन्फ़िगरेशन फ़ाइल .\config.json पर मौजूद है।


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML प्रयोग तैयार करें

आइए उस कार्यक्षेत्र में एक प्रयोग बनाते हैं जिसे हमने अभी प्रारंभ किया है, जिसका नाम 'aml-experiment' है।


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## एक कंप्यूट क्लस्टर बनाएं  
आपको अपने AutoML रन के लिए एक [कंप्यूट टारगेट](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) बनाना होगा।  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## डेटा  
सुनिश्चित करें कि आपने डेटासेट को Azure ML पर अपलोड कर दिया है और कि कुंजी का नाम डेटासेट के नाम के समान है।


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## सबसे अच्छा मॉडल डिप्लॉय करें

सबसे अच्छा मॉडल डिप्लॉय करने के लिए निम्नलिखित कोड चलाएँ। आप Azure ML पोर्टल में डिप्लॉयमेंट की स्थिति देख सकते हैं। यह प्रक्रिया कुछ मिनटों का समय ले सकती है।


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## एंडपॉइंट का उपयोग करें
आप निम्नलिखित इनपुट नमूने में इनपुट जोड़ सकते हैं।


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
